Check we have connection correct. To test this we call get a FHIR Conformance statement from the server

In [9]:
import requests
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os

fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")
server = os.getenv("FHIR_SERVER")

api_url = server + "metadata"
response = requests.get(api_url)
#response.json()

Find a patient. Simple search for a patient named wrexham

In [10]:


api_url = server + "Patient?name=wrexham"
print(api_url)
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

https://gen-hie-prod.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Patient?name=wrexham


{'resourceType': 'Bundle',
 'id': '68cf4730-5f43-4fbb-83b3-0203e4370d26',
 'type': 'searchset',
 'timestamp': '2025-12-24T07:25:47Z',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://gen-hie-prod.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Patient?name=wrexham'}]}

Now lets find diagnostic reports for this patient.

In [11]:
api_url = server + "DiagnosticReport?patient=18"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': 'b81f9b00-e031-4f7a-a29a-7f884df30b84',
 'type': 'searchset',
 'timestamp': '2025-12-24T07:25:53Z',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://gen-hie-prod.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/DiagnosticReport?patient=18'}]}

Can also return details about the organisation that created this.

In [12]:
api_url = server + "Organization/5"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Organization',
 'active': True,
 'id': '5',
 'identifier': [{'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': 'RX1'}],
 'name': 'NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST',
 'partOf': {'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': 'QT1'},
  'reference': 'Organization/6'},
 'type': [{'coding': [{'code': 'RO197',
     'display': 'NHS TRUST',
     'system': 'https://digital.nhs.uk/services/organisation-data-service/CodeSystem/ODSOrganisationRole'}]}],
 'meta': {'lastUpdated': '2025-12-19T17:19:21Z', 'versionId': '2'}}

Example using python to interpret the json and also using [fhirclient](https://github.com/smart-on-fhir/client-py)

In [57]:
import fhirclient.models.diagnosticreport as dr
import pandas as pd

api_url = server + "DiagnosticReport?_count=50"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
responseJSON = response.json()

print(responseJSON['total'])

reports = []
reportJ = []
for entry in responseJSON['entry']:
    #print(entry['resource'])
    print(entry['resource']['resourceType'], entry['resource']['issued'] )
    report = dr.DiagnosticReport(entry['resource'])
    reportJ.append(entry['resource'])
    reports.append(report)
    for coding in report.code.coding:
        print(coding.code)

6
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-12-17T12:22:52+00:00
SANGER-Family-follow-up-Rare
R375.1
DiagnosticReport 2025-12-17T12:25:36+00:00
M80_AML_GX
M80.2
DiagnosticReport 2025-12-18T12:14:40+00:00
PCR-FLT3-ITD
M80.18
M80.21


In [58]:
print(len(reports))
df1 = pd.DataFrame([vars(s) for s in reports])
df1

6


,basedOn,category,code,conclusion,conclusionCode,effectiveDateTime,effectivePeriod,encounter,identifier,imagingStudy,...,extension,modifierExtension,text,id,implicitRules,language,meta,_server,_resolved,_owner
0,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,7,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None
1,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,16,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None
2,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,22,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None
3,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,61,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None
4,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,75,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None
5,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,<fhirclient.models.fhirreference.FHIRReference...,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,88,None,None,<fhirclient.models.meta.Meta object at 0x10db3...,None,None,None


In [59]:
def performer(my_list):
    performr = ""
    for item in my_list:
        performr = item.display
    return performr

def codeCode(concept):
    code = ""
    for coding in concept.coding:
        code = coding.code

    return code
def codeDisplay(concept):
    code = ""
    for coding in concept.coding:
        code = coding.display

    return code

df1['performerDisplay'] = df1['performer'].apply(performer)
df1['codingCode'] = df1['code'].apply(codeCode)
df1['codingDisplay'] = df1['code'].apply(codeDisplay)

df1

,basedOn,category,code,conclusion,conclusionCode,effectiveDateTime,effectivePeriod,encounter,identifier,imagingStudy,...,id,implicitRules,language,meta,_server,_resolved,_owner,performerDisplay,codingCode,codingDisplay
0,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,7,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None,NHS North West Genomics,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury..."
1,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,16,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None,NHS North West Genomics,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury..."
2,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,22,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None,NHS North West Genomics,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury..."
3,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,61,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None,NHS North West Genomics,R375.1,TEST: Sanger Sequencing - Family follow-up tes...
4,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,75,None,None,<fhirclient.models.meta.Meta object at 0x10dad...,None,None,None,NHS North West Genomics,M80.2,"Acute Myeloid Leukaemia, Multi-target NGS pane..."
5,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,<fhirclient.models.fhirreference.FHIRReference...,[<fhirclient.models.identifier.Identifier obje...,None,...,88,None,None,<fhirclient.models.meta.Meta object at 0x10db3...,None,None,None,NHS North West Genomics,M80.21,"Acute Myeloid Leukaemia, FLT3 TKD hotspot"
